In [200]:
import string
import torch.nn as nn
import torch.nn.functional as F

# A simple char tokenizer
class Tokenizer: 
    
    def __init__(self, data):
        unique_chars = set(data)
        chars = sorted(list(unique_chars))        
        self.vocab_size = len(unique_chars)
        self.encoding_map = {c:i for i, c in enumerate(chars)}
        self.decoding_map = {i:c for i, c in enumerate(chars)}

        
    def encode(self, text): 
        return [self.encoding_map[c] for c in text]
        
    def decode(self, idx):
        return ''.join([self.decoding_map[i] for i in idx])

tokenizer = Tokenizer(string.ascii_lowercase)
tokenizer.decode(tokenizer.encode('hello'))

'hello'

In [201]:
# Read in our dataset, tokenize and split into train/test
with open('meditations.txt', 'r') as f:
    data = f.read()
n = len(data)
tokenizer = Tokenizer(data)
tokenized_data = torch.tensor(tokenizer.encode(data), dtype=torch.long)

train_data = tokenized_data[:int(n*0.9)]
val_data = tokenized_data[int(n*0.9):]

print(f'Num training tokens: {train_data.size(dim=0)}')
print(f'Num test tokens: {val_data.size(dim=0)}')
print(f'Vocab size: {tokenizer.vocab_size}')

Num training tokens: 218013
Num test tokens: 24224
Vocab size: 67


In [244]:
embedding_size = 32
vocab_size = tokenizer.vocab_size # Should be 67
context_size = 8
device = 'cuda' if torch.cuda.is_available() else 'mps'
batch_size = 64
eval_iters = 200
learning_rate = 3e-3
max_iters = 5000
eval_interval = 500


# Generates (B, T) tensors for x and y
# Each batch is a sequence of length T of tokens where y_i is x_i offset by 1
def get_batch(data, batch_size): 
    ix = torch.randint(len(data)-context_size, (batch_size, )) 
    x = torch.stack([data[i:i+context_size] for i in ix])
    y = torch.stack([data[i+1:i+context_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y
    
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    splits = {'train': train_data, 'val': val_data }
    for split, data in splits.items():
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(data, batch_size)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

# Let's train! 
class VerySimpleLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(num_embeddings=vocab_size, embedding_dim=vocab_size)        
        
    # idx is a (B, T) sized tensor 
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # B, T, embed_size
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def sample(self, idx, length):
        # idx: (B, T)
        for _ in range(length):
            idx_cond = idx[:, -context_size:] # make sure we are only inputting context up to context_size
            logits, loss = m(idx_cond) # logits is a (B, T, C) tensor with raw output of the model for each token
            logits = logits[:, -1, :] # Let's only select the prediction for the last token
            probs = F.softmax(logits, dim=-1) # Get our probabilities (B, C) vector
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) 
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    

        
model = VerySimpleLanguageModel()
m = model.to(device)
x, y = get_batch(train_data, batch_size)


print(sum(p.numel() for p in m.parameters()), 'parameters')
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    # Grab a batch 
    xb, yb = get_batch(train_data, batch_size)

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

4489 parameters
step 0: train loss 4.4898, val loss 4.5193
step 500: train loss 3.0807, val loss 3.0974
step 1000: train loss 2.5844, val loss 2.5925
step 1500: train loss 2.4416, val loss 2.4464
step 2000: train loss 2.3989, val loss 2.3955
step 2500: train loss 2.3728, val loss 2.3749
step 3000: train loss 2.3522, val loss 2.3584
step 3500: train loss 2.3552, val loss 2.3621
step 4000: train loss 2.3412, val loss 2.3469
step 4500: train loss 2.3382, val loss 2.3447
step 4999: train loss 2.3415, val loss 2.3426


In [246]:
context = torch.zeros((1, 1), dtype=torch.long,  device=device)
print(tokenizer.decode(m.sample(context, length=500)[0].tolist())) # Not very intelligble but doesn't just output shit


powho thytutave po bliJusanavesans dXnver
beexivivichime than k ng is; th h tes
wngengsthengassan ilicheccching, a mbut n the; o e ndecqus alveee Ins
t ald t be, f ay nlle pit ber
he oke erntin
ad unous,waman. g brme,), tho t pll son nd
Thenthaling
it harathan hePlexid an (ot d t f aprrino hoblend bud the, a ws.

ixciove
ys? try) f ns t fulins thinst t aco
then t oro they it oojunk ay wo toreanis mplirchas ara the acheriend man ulin d pplto ale il forer becosar id he t anhithime o
thececotim

th


In [279]:
embedding_size = 32
vocab_size = tokenizer.vocab_size # Should be 67
context_size = 8
device = 'cuda' if torch.cuda.is_available() else 'mps'
batch_size = 32
eval_iters = 200
learning_rate = 1e-3
max_iters = 5000
eval_interval = 500
n_head = 4
n_layer = 1
dropout = 0.2


# Generates (B, T) tensors for x and y
# Each batch is a sequence of length T of tokens where y_i is x_i offset by 1
def get_batch(data, batch_size): 
    ix = torch.randint(len(data)-context_size, (batch_size, )) 
    x = torch.stack([data[i:i+context_size] for i in ix])
    y = torch.stack([data[i+1:i+context_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y
    
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    splits = {'train': train_data, 'val': val_data }
    for split, data in splits.items():
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(data, batch_size)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

class FeedForward(nn.Module): 
    
    def __init__(self, embedding_size): 
        super().__init__()
        self.linear = nn.Linear(embedding_size, 4 * embedding_size)
        self.relu = nn.ReLU()
        self.second_linear = nn.Linear(4 * embedding_size, embedding_size)
        self.m = nn.Sequential(self.linear, self.relu, self.second_linear, nn.Dropout(dropout))
        
        
    def forward(self, x): 
        return self.m(x)


class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(embedding_size, head_size, bias=False)
        self.query = nn.Linear(embedding_size, head_size, bias=False)
        self.value = nn.Linear(embedding_size, head_size, bias=False)
        
        # This is a buffer and not a regular parameter since we don't need to update this during backprop
        self.register_buffer('tril', torch.tril(torch.ones(context_size, context_size)))
        self.dropout = nn.Dropout(dropout)
        
    # x is of size B, T, embedding_size
    # output is B, T, head_size
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x) 
        
        # q x k scaled by sqrt(head_size)
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) 
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        
        # weighted aggregation of v 
        v = self.value(x) # B, T, head_size
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out
    
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, embedding_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
    
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x
        
        
        
# Let's train! 
class AureliusGPT(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, embedding_size)
        self.token_position_embedding_table = nn.Embedding(context_size, embedding_size)
        self.blocks = nn.Sequential(*[Block(embedding_size, n_head=n_head) for _ in range(n_layer)])
        self.lm_head = nn.Linear(embedding_size, vocab_size)
        self.ln_f = nn.LayerNorm(embedding_size)

        
    # idx is a (B, T) sized tensor 
    def forward(self, idx, targets=None):
        B, T = idx.shape # Get the batch size and the current context size
        
        tok_emb = self.token_embedding_table(idx) # B, T, embed_size
        
        # get all the position embeddings for each position in our context
        pos_emb = self.token_position_embedding_table(torch.arange(T, device=device)) # B, T, embed_size
        x = tok_emb + pos_emb # B, T, embed_size
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def sample(self, idx, length):
        # idx: (B, T)
        for _ in range(length):
            idx_cond = idx[:, -context_size:] # make sure we are only inputting context up to context_size
            logits, loss = m(idx_cond) # logits is a (B, T, C) tensor with raw output of the model for each token
            logits = logits[:, -1, :] # Let's only select the prediction for the last token
            probs = F.softmax(logits, dim=-1) # Get our probabilities (B, C) vector
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) 
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    

        
model = AureliusGPT()
m = model.to(device)
x, y = get_batch(train_data, batch_size)


print(sum(p.numel() for p in m.parameters()), 'parameters')
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    # Grab a batch 
    xb, yb = get_batch(train_data, batch_size)

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

17283 parameters
step 0: train loss 4.3466, val loss 4.3481
step 500: train loss 2.3202, val loss 2.3270
step 1000: train loss 2.2104, val loss 2.2171
step 1500: train loss 2.1378, val loss 2.1534
step 2000: train loss 2.0998, val loss 2.1054
step 2500: train loss 2.0699, val loss 2.0573
step 3000: train loss 2.0386, val loss 2.0377
step 3500: train loss 2.0320, val loss 2.0289
step 4000: train loss 2.0078, val loss 2.0122
step 4500: train loss 1.9902, val loss 1.9947
step 4999: train loss 1.9807, val loss 1.9819


In [281]:
context = torch.zeros((1, 1), dtype=torch.long,  device=device)
print(tokenizer.decode(m.sample(context, length=500)[0].tolist())) # Not very intelligble but doesn't just output shit


antrey reads ranythoughingnear whaie A blamesst andyse bo vhiy his sues sue thoseppectygrasto thet ngenfied iflich fam not, mone. 
thave ithat it to mings thou a wht
of in tnfod of abit a monen of man (phep berolown turt for ad and be diones st fatieds tacon, Proung
it thowe waccht lowd
shosen thou wroagle,
al
cestimenit a urereail no
who, rus things many thou buty ity, the in ines sato broe otatemve are? doe frelvated pre, I. I mian all, thou and rivery and aracart, dot tharter matet etity alt 
